In [1]:
import os
os.environ["TRANSFORMERS_CACHE"] = "/workspace/cache/"
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GenerationConfig
import transformers
import torch
import time

In [2]:
model = "tiiuae/falcon-7b"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
)

#model = AutoModelForCausalLM.from_pretrained(model,
#        #load_in_4bit=True,
#        device_map='auto',
#        torch_dtype=torch.float16,
#        trust_remote_code=True,
#        quantization_config=BitsAndBytesConfig(
#            load_in_4bit=True,
#            bnb_4bit_compute_dtype=torch.bfloat16,
#            bnb_4bit_use_double_quant=True,
#            bnb_4bit_quant_type='nf4'
#        ),
#)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The model 'RWForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerNormForC

In [6]:
now = time.time()
sequences = pipeline(
   "What is ",
    max_new_tokens=500,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)

for seq in sequences:
    print(f"Result: {seq['generated_text']}")
    

duration = time.time() - now
tks=50/duration
print(f"{tks} tokens/s")
print(f"{1/tks*1000} ms/token")

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Result: What is ‘Gut Microbiome’ and Why is it Important?
The microbiome is a term used to describe all living microorganisms, or bacteria, that live in or on our bodies. These microbes are present everywhere and are found all over the body — from our skin to our hair to our intestines.
The term microbiome is often used interchangeably with the term gut microbiome and is commonly associated with the gastrointestinal tract. It’s the name we give to all of the microbes and bacteria that exist in our large intestine and colon. The gut microbiome is home to trillions of microorganisms. It’s estimated that there are more than 100 trillion microbes in the human body, with at least 40 trillion of those microbes living in the gut alone.
The bacteria found within our gut are essential for many reasons — they help us to break down our food and absorb important nutrients, they regulate our immune system functions, and they protect the body from infection. It’s no surprise that we have an intimate

In [ ]:
inputs = tokenizer("""Texte:Chapitre 13 Les composantes de la légalité
Préfixe:Chapitre 13
###
Texte:2.La réduction des marges de manœuvre budgétaire
Préfixe:2.
###
Texte:III.	Sources du droit pénal
Préfixe:III.
###
Texte:Partie 3: Décentralisation et finances publiques
Préfixe:Partie 3:
###
Texte:A.L’aspect matériel
Préfixe:""", return_tensors="pt")
input_ids = inputs.input_ids.to("cuda")

In [ ]:
generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.9,
    num_beams=4
)
now = time.time()
with torch.no_grad():
    generation_output = model.generate(
        input_ids=input_ids,
        #generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=100,
    )
duration = time.time() - now
try:
    s = generation_output.sequences[0]
except:
    s = generation_output[0]

tks = (s.shape[0] - input_ids.shape[1])/duration
print(f"{tks} tokens/s")
print(f"{1/tks*1000} ms/token")
output = tokenizer.decode(s)
print(output)
#res = prompter.get_response(output)
#print(res)